In [ ]:
%pip install semver

In [ ]:
import semver
import shutil
import os
import json


In [ ]:
def read_value_from_json(file_path, key):
    with open(file_path, 'r') as file:
        data = json.load(file)
        value = data[key]
        return value
    
def read_values_from_json(file_path, keys):
    with open(file_path, 'r') as file:
        data = json.load(file)
        values = []
        for key in keys:
            values.append(data[key])
        return values
    
# ver = read_value_from_json('src/info.json', 'version')
# appname = read_value_from_json('src/info.json', 'name')
# print(ver, appname)

In [ ]:
def write_kv_to_json(file_path, key, value):
    with open(file_path, 'r') as file:
        data = json.load(file)
        data[key] = value
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

def write_kvs_to_json(file_path, keys, values):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for i in range(len(keys)):
            data[keys[i]] = values[i]
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

# write_kv_to_json('src/info.json', 'version', '0.0.5')
# write_kv_to_json('src/info.json', 'version', '3.14.159')
            

In [ ]:
def zip_app():
    appname, version = read_values_from_json('mod/src/info.json', ['name', 'version'])
    zip_filename = f'{appname}_{version}'
    zip_path = f'{zip_filename}'
    shutil.make_archive(zip_path, 'zip', 'mod')
    return zip_path, appname


In [ ]:
def bump_version(file_path, type='patch'):
    current_version = read_value_from_json(file_path, 'version')
    if type == 'major':
        new_version = semver.bump_major(current_version)
    elif type == 'minor':
        new_version = semver.bump_minor(current_version)
    elif type == 'patch':
        new_version = semver.bump_patch(current_version)
    else:
        raise ValueError('Invalid type')
    write_kv_to_json(file_path, 'version', new_version)
    return new_version

# bump_version('src/info.json')


In [ ]:
def find_other_installed_versions(mod_folder):
    files = os.listdir(mod_folder)
    appname, version = read_values_from_json('mod/src/info.json', ['name', 'version'])
    other_versions = [f for f in files if appname in f and version not in f]
    return other_versions

# find_other_installed_versions(factorio_mods)

In [ ]:
def remove_old_versions(mod_folder):
    other_versions = find_other_installed_versions(mod_folder)
    for version in other_versions:
        os.remove(mod_folder + version)
    return other_versions

# remove_old_versions(factorio_mods)

In [ ]:
def package_app(mod_folder):
    version = bump_version('mod/src/info.json')
    path, appname = zip_app()
    dest = shutil.copy(f'{path}.zip', mod_folder)
    old_versions = remove_old_versions(mod_folder)
    return f'Packaged {appname} {version} to {dest} and removed {old_versions}'


In [ ]:
# factorio_mods = 'C:/Users/peter/AppData/Roaming/Factorio/mods/'
# package_app(factorio_mods)